# 🔹 Qué hace este bloque
- Recorre todas las categorías.
- Extrae las categorias.

In [ ]:
import requests # Bibliteca para hacer peticiones para abrir una pagina web desde mi codigo
from bs4 import BeautifulSoup # Herramienta para parsear HTML y extraer datos de forma amigable


# Defino la URL  del sitio principal
BASE_URL = "https://books.toscrape.com/"

#  Te lleva a la página donde están listadas todas las categorías.
CATEGORIAS_URL = BASE_URL + "index.html" 

def extraer_categorias():
    # Abre la pagina como si fuera un navegador y devuelve el contenido en formato HTML
    respuesta = requests.get(CATEGORIAS_URL)
    soup = BeautifulSoup(respuesta.text, "html.parser")# Convierto ese HTML en un objeto manipulable

    # Usa selectores CSS para encontrar elementos en el HTML
    categorias_links = soup.select("ul.nav-list ul li a") #etiqueta

    categoria = {} # Creo un diccionario vacio para guardar categorias

    for link in categorias_links: 
        nombre = link.text.strip() # Obtiene el link de la categoria y le quita espacios extra
        url_relativa = link['href'] # Saco la URL relativa 
        url_completa = BASE_URL + url_relativa # Armo la URL completa que te permiten navegar a cada categoría individualmente.
        categoria[nombre] = url_completa # Guardo en el diccionario: nombre -> URL

    print(f"Categorias encontradas: {len(categoria)}\n")
    return categoria
   

categoria = extraer_categorias()

Categorias encontradas: 50



# Creación de Base de Datos

In [ ]:
import sqlite3

# Si el archivo no existe, lo crea. Si ya existe lo abre
conn = sqlite3.connect("libros.db") # Crea el archivo libros.db
cursor = conn.cursor() # Cursor es el "lapiz" con el que escribir y lees dentro de la base de datos

# execute -> ejecuta instrucción SQL dentro de la BD
cursor.execute("""
-- Evita errores si la tabla ya fue creada antes
CREATE TABLE IF NOT EXISTS categorias( 
    id INTEGER PRIMARY KEY AUTOINCREMENT, 
    nombre TEXT UNIQUE  -- UNIQUE Asegura que los valores de una columna no se repitan 
)
""")

cursor.execute("""
               
CREATE TABLE IF NOT EXISTS autores(
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    nombre TEXT UNIQUE
)
""")
cursor.execute("""
CREATE TABLE IF NOT EXISTS libros(
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    titulo TEXT,
    precio REAL,   --Tipo de dato numerico que representa numeros decimales o enteros 
    rating TEXT,
    disponibilidad TEXT,
    link TEXT,
    UPC TEXT,       --UPC-> Codigo universal del producto
    descripcion TEXT,
    FOREIGN KEY (categoria_id) REFERENCES categorias(id)                                
)
""")
try:
     cursor.execute("ALTER TABLE libros ADD COLUMN categoria_id TEXT")
except sqlite3.OperationalError: # Problemas operativos con la BD, (INSERTAR columna que ya existe)
     print("La columna 'categoria_id' ya existe.")



cursor.execute("""
CREATE TABLE IF NOT EXISTS libro_autor(
    libro_id INTEGER,
    autor_id INTEGER,
    PRIMARY KEY (libro_id, autor_id),
    FOREIGN KEY (libro_id) REFERENCES libros(id),
    FOREIGN KEY (autor_id) REFERENCES autores(id)
)
""")
cursor.execute("PRAGMA table_info(libros)") # Muestra la estructura de la tabla libros
print(cursor.fetchall())#Recupera todos los resultados y los imprime 
conn.commit() # Guarda todos los cambios realizados en la BD 
conn.close()

La columna 'categoria_id' ya existe.
[(0, 'id', 'INTEGER', 0, None, 1), (1, 'titulo', 'TEXT', 0, None, 0), (2, 'categoria', 'TEXT', 0, None, 0), (3, 'precio', 'REAL', 0, None, 0), (4, 'rating', 'TEXT', 0, None, 0), (5, 'descripcion', 'TEXT', 0, None, 0), (6, 'disponibilidad', 'TEXT', 0, None, 0), (7, 'link', 'TEXT', 0, None, 0), (8, 'UPC', 'TEXT', 0, None, 0), (9, 'categoria_id', 'TEXT', 0, None, 0)]


# Conexion a la BD.
- Insertar categorias
- Insertar autores
- Insertar libros
- Relación libros con autor


In [ ]:
import sqlite3

# Conexión a la BD
def conectar():
    return sqlite3.connect("libros.db")

# Guardar categoría
def guardar_categoria(nombre):
    conn = conectar()
    cursor = conn.cursor()
    # INSERT OR IGNORE -> Inserta la categoria solo si no existe
    cursor.execute("INSERT OR IGNORE INTO categorias(nombre) VALUES(?)", (nombre,))
    conn.commit()
    cursor.execute("SELECT id FROM categorias WHERE nombre=?", (nombre,)) # ?-> placeholdeer :espacio reservado dentro de una consulta SQL
    categoria_id = cursor.fetchone()[0] # - recupera la primera fila del resultado y extraer el primer valor
    conn.close()

    return categoria_id

# Guardar autor
def guardar_autor(nombre):
    conn = conectar()
    cursor = conn.cursor()
    cursor.execute("INSERT OR IGNORE INTO autores(nombre) VALUES(?)", (nombre,))
    conn.commit()
    cursor.execute("SELECT id FROM autores WHERE nombre=?", (nombre,))
    autor_id = cursor.fetchone()[0]
    conn.close()
    return autor_id

# Guardar libro
def guardar_libro(titulo, precio, rating, disponibilidad, link, UPC, categoria_id,descripcion):
    conn = conectar()
    cursor = conn.cursor()
    cursor.execute("""
        INSERT INTO libros(titulo, precio, rating, disponibilidad, link, UPC, categoria_id,descripcion)
        VALUES(?,?,?,?,?,?,?,?)
    """, (titulo, precio, rating, disponibilidad, link, UPC, categoria_id, descripcion))
    conn.commit()
    libro_id = cursor.lastrowid # Devuelve el id de la ultima fila insertada por ese cursor
    conn.close()
    return libro_id

# Relacionar libro con autor
def guardar_libro_autor(libro_id, autor_id):
    conn = conectar()
    cursor = conn.cursor()
    cursor.execute("INSERT OR IGNORE INTO libro_autor(libro_id, autor_id) VALUES(?,?)", (libro_id, autor_id))
    conn.commit()
    conn.close()



# Consultas Google Books API

In [ ]:
import requests
from bs4 import BeautifulSoup

# --- Scraping en BooksToScrape (por autor) ---
def obtener_autor_books_to_scrape(url_libro):
    try:
        response = requests.get(url_libro)
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')
            info_table = soup.select('table.table.table-striped tr')
            if len(info_table) > 1: # si hay mas de una fila
                autor = info_table[1].text.strip() # toma la segunda que suele contener el autor, limpia espacion y devuelve
                return autor
    except Exception as e:
        print(f"Error en scraping: {e}")
    return "Autor desconocido"


# --- API de Google Books ---
def buscar_libro(titulo):
    url = "https://www.googleapis.com/books/v1/volumes"
    params = {
        "q": f"intitle:{titulo}", # busca por titulo
        "maxResults": 1, # limita a 1 resultado(para evitar multiples coincidencias)
        "langRestrict": "es" # filtra por idioma español
    }

    respuesta = requests.get(url, params=params) #Hace la petición con los parámetros definidos

    if respuesta.status_code == 200:
        datos = respuesta.json()
        if "items" in datos:
            info = datos["items"][0]["volumeInfo"] # Si hay resultados,toma el primero y accede a su info

            titulo_libro = info.get("title", "Sin título")
            autores = ", ".join(info.get("authors", ["Autor desconocido"]))
            descripcion = info.get("description", "Sin descripción")

            return {
                "Título": titulo_libro,
                "Autores": autores,
                "Descripción": descripcion
            }

    # Si no encuentra en la API, intenta con BooksToScrape
    print("No se encontró en Google Books, intentando en BooksToScrape...")
    autor_scrape = obtener_autor_books_to_scrape("http://books.toscrape.com/catalogue/a-light-in-the-attic_1000/index.html")

    return {
        "Título": "No encontrado",
        "Autores": autor_scrape,
        "Descripción": "No disponible"
    }


# --- Ejemplo de uso ---
print(buscar_libro("A Light in the Attic"))


{'Título': 'A Light in the Attic', 'Autores': 'Shel Silverstein', 'Descripción': "NOW AVAILABLE AS AN EBOOK! From New York Times bestselling author Shel Silverstein, the creator of the beloved poetry collections Where the Sidewalk Ends, Falling Up, and Every Thing On It, comes an imaginative book of poems and drawings—a favorite of Shel Silverstein fans young and old. This digital edition also includes twelve poems previously only available in the special edition hardcover. A Light in the Attic delights with remarkable characters and hilariously profound poems in a collection readers will return to again and again. Here in the attic you will find Backward Bill, Sour Face Ann, the Meehoo with an Exactlywatt, and the Polar Bear in the Frigidaire. You will talk with Broiled Face, and find out what happens when Somebody steals your knees, you get caught by the Quick-Digesting Gink, a Mountain snores, and They Put a Brassiere on the Camel. Come on up to the attic of Shel Silverstein and let

In [5]:
# Función para obtener detalle completo de un libro
def obtener_detalle_libro(url_libro, categoria):
    try:
        response = requests.get(url_libro)
        soup = BeautifulSoup(response.text, "html.parser")
        
        titulo = soup.select_one(".product_main h1").get_text(strip=True)
        precio_text = soup.select_one(".price_color").get_text(strip=True)
        
        # Limpiar precio de manera más robusta
        try:
            precio_limpio = precio_text.replace('£', '').replace('Â', '').replace('€', '').strip()
            precio = float(precio_limpio)
        except ValueError:
            precio = 0.0  # Precio por defecto si no se puede convertir
            
        
        
        disponibilidad = soup.select_one(".instock.availability").get_text(strip=True)
        
        # Rating (está en class="star-rating X")
        rating_tag = soup.select_one(".star-rating")
        rating = rating_tag["class"][1] if rating_tag else "Sin rating"

        # UPC (está en la tabla de detalles)
        info_table = soup.select("table.table.table-striped tr")
        upc = info_table[0].td.get_text(strip=True) if len(info_table) > 0 else "Sin UPC"
        
        # Descripción del producto
        descripcion_tag = soup.select_one("#product_description ~ p")
        descripcion = descripcion_tag.get_text(strip=True) if descripcion_tag else "Sin descripción"

        # Buscar autor usando Google Books API
        info_google = buscar_libro(titulo)
        autor = info_google.get("Autores", "Autor desconocido")
        
        # Si Google no tiene descripción, usar la del sitio
        if info_google.get("Descripción") != "Sin descripción":
            descripcion = info_google.get("Descripción", descripcion)

        return {
            "titulo": titulo,
            "autor": autor,
            "precio": precio,
            "rating": rating,
            "disponibilidad": disponibilidad,
            "categoria": categoria,
            "link": url_libro,
            "UPC": upc,
            "descripcion": descripcion
        }
    except Exception as e:
        print(f"Error al obtener {url_libro}: {e}")
        return None

#  OBTENER LIBROS POR CATEGORIAS

In [ ]:
import time # Para no hacer pedidos(requests) una tras otra, porque puede causar problemas

"""Creo una función y le doy dos argumentos"""
def obtener_libros_por_categoria(nombre_categoria,url_categoria):

    
    pagina_actual = url_categoria #Asigna la URL inicial de la categoría a una variable que iremos actualizando
    libros = [] # Lista vacía en donde se va guardar un diccionario por cada libro

    while True:
        print(f"Scrapeando {pagina_actual}...")
        respuesta = requests.get(pagina_actual)
        soup = BeautifulSoup(respuesta.text, "html.parser")

        libros_en_pagina = soup.select(".product_pod")
        
        for libro in libros_en_pagina:
            titulo = libro.h3.a["title"]
            precio = libro.select_one(".price_color").text.strip()
            rating = libro.p["class"][1] # Investigar

            url_relativa = libro.h3.a["href"]
            url_libro = "/".join(pagina_actual.split("/")[:-1]) + "/" + url_relativa # investigar

            

            libros.append({
                "categoria": nombre_categoria,
                "titulo": titulo,
                "precio": precio,
                "rating": rating,
                "url_libro": url_libro,
               
            })


        next_button = soup.select_one("li.next a") # Busca si hay un enlace a la próxima página. Si existe, sigue
        # Construye la URL de la próxima página
        if next_button:
            next_url = next_button["href"]
            # Si pagina_actual = ".../index.html" y next_url = "page-2.html"➡ Reemplaza index.html con page-2.html
            if "index.html" in pagina_actual:
                pagina_actual = pagina_actual.replace("index.html", next_url )
            else:
                # Esta línea se ejecuta cuando hay más de una página en la categoría (ejemplo: page-2.html, page-3.html, etc.).
                pagina_actual = "/".join(pagina_actual.split("/")[:-1])+ "/" + next_url
        else: 
            break

        time.sleep(1) # Espera 1 segundo antes de pasar a la próxima página
    return libros # Devuelve la lista de diccionarios con todos los libros encontrados
#  lista vacía para juntar todos los libros del sitio
todos_los_libros = []
# Recorre todas las categorías que ya tenemos en el diccionario
for nombre,url in categoria.items():
    print(f"Scrapeando categoría: {nombre}...")

    libros_categoria = obtener_libros_por_categoria(nombre,url)

    todos_los_libros.extend(libros_categoria)  # agregamos todos los libros a la lista general
print(f"\nScraping completado. Total de libros encontrados: {len(todos_los_libros)}")

Scrapeando categoría: Travel...
Scrapeando https://books.toscrape.com/catalogue/category/books/travel_2/index.html...
Scrapeando categoría: Mystery...
Scrapeando https://books.toscrape.com/catalogue/category/books/mystery_3/index.html...
Scrapeando https://books.toscrape.com/catalogue/category/books/mystery_3/page-2.html...
Scrapeando categoría: Historical Fiction...
Scrapeando https://books.toscrape.com/catalogue/category/books/historical-fiction_4/index.html...
Scrapeando https://books.toscrape.com/catalogue/category/books/historical-fiction_4/page-2.html...
Scrapeando categoría: Sequential Art...
Scrapeando https://books.toscrape.com/catalogue/category/books/sequential-art_5/index.html...
Scrapeando https://books.toscrape.com/catalogue/category/books/sequential-art_5/page-2.html...
Scrapeando https://books.toscrape.com/catalogue/category/books/sequential-art_5/page-3.html...
Scrapeando https://books.toscrape.com/catalogue/category/books/sequential-art_5/page-4.html...
Scrapeando cat

In [ ]:
# LIMPIAR Y GUARDAR LOS DATOS EN LA BASE DE DATOS
print("\n💾 Guardando datos en la base de datos...")

contador_guardados = 0

for libro in todos_los_libros:
    try:
        # Limpiar el precio
        precio_texto = libro['precio']
        precio_limpio = precio_texto.replace('£', '').replace('Â', '').strip()
        precio_float = float(precio_limpio)
        
        # Guardar categoría si no existe y obtener ID
        categoria_id = guardar_categoria(libro['categoria'])
        
        # Guardar libro en BD
        libro_id = guardar_libro(
            libro['titulo'],
            precio_float,  # Precio ya convertido a float
            libro['rating'],
            "In stock",  # Disponibilidad por defecto
            libro['url_libro'],
            "N/A",  # UPC no disponible en listado # investigar
            categoria_id,
            "Descripción no disponible"  # Descripción por defecto
        )
        
        # Para el autor, usar un nombre genérico por ahora
      # investigar
        autor_id = guardar_autor("Autor por determinar")
        guardar_libro_autor(libro_id, autor_id)
        
        contador_guardados += 1
        
        if contador_guardados % 100 == 0:
            print(f"   📚 {contador_guardados} libros guardados...")
            
    except Exception as e:
        print(f"   ❌ Error guardando '{libro['titulo']}': {e}")
        continue

print(f"\n🎉 GUARDADO COMPLETADO!")
print(f"📊 {contador_guardados} libros guardados exitosamente")
print(f"🗃️ Base de datos 'libros.db' lista para las consultas")


💾 Guardando datos en la base de datos...
   📚 100 libros guardados...
   📚 200 libros guardados...
   📚 300 libros guardados...
   📚 400 libros guardados...
   📚 500 libros guardados...
   📚 600 libros guardados...
   📚 700 libros guardados...
   📚 800 libros guardados...
   📚 900 libros guardados...
   📚 1000 libros guardados...

🎉 GUARDADO COMPLETADO!
📊 1000 libros guardados exitosamente
🗃️ Base de datos 'libros.db' lista para las consultas


#  EJECUCION DE SCRAPING COMPLETO

In [7]:
# SCRAPING COMPLETO CON GUARDADO EN BASE DE DATOS
def ejecutar_scraping_completo():
    """
    Ejecuta el scraping completo del sitio y guarda todo en la base de datos
    """
    print("🕷️ Iniciando misión: Infiltración en Books To Scrape...")
    
    # Obtener todas las categorías
    categorias = extraer_categorias()
    
    contador_libros = 0
    
    # Recorrer cada categoría
    for nombre_categoria, url_categoria in categorias.items():
        print(f"\n📚 Scrapeando categoría: {nombre_categoria}")
        
        # Guardar categoría en BD y obtener su ID
        categoria_id = guardar_categoria(nombre_categoria)
        
        # Obtener libros básicos de la categoría
        libros_categoria = obtener_libros_por_categoria(nombre_categoria, url_categoria)
        
        # Procesar cada libro para obtener detalles completos
        for libro_basico in libros_categoria:
            print(f"  📖 Procesando: {libro_basico['titulo']}")
            
            # Obtener detalles completos del libro
            detalle = obtener_detalle_libro(libro_basico['url_libro'], nombre_categoria)
            
            if detalle:
                # Guardar libro en BD
                libro_id = guardar_libro(
                    detalle['titulo'],
                    detalle['precio'],
                    detalle['rating'],
                    detalle['disponibilidad'],
                    detalle['link'],
                    detalle['UPC'],
                    categoria_id,
                    detalle['descripcion']
                )
                
                # Procesar autores (pueden ser múltiples separados por coma)
                autores = detalle['autor'].split(', ')
                for autor_nombre in autores:
                    autor_nombre = autor_nombre.strip()
                    if autor_nombre and autor_nombre != "Autor desconocido":
                        # Guardar autor y relacionar con libro
                        autor_id = guardar_autor(autor_nombre)
                        guardar_libro_autor(libro_id, autor_id)
                
                contador_libros += 1
                
                # Pequeña pausa para no sobrecargar el servidor
                time.sleep(0.5)
        
        print(f"✅ Categoría {nombre_categoria} completada")
        time.sleep(1)  # Pausa entre categorías
    
    print(f"\n🎉 MISIÓN COMPLETADA!")
    print(f"📊 Total de libros procesados: {contador_libros}")
    print(f"🗃️ Base de datos 'libros.db' lista para la acción")
ejecutar_scraping_completo()
# Ejecutar el scraping completo
# ¡CUIDADO! Esto puede tomar bastante tiempo


🕷️ Iniciando misión: Infiltración en Books To Scrape...
Categorias encontradas: 50


📚 Scrapeando categoría: Travel
Scrapeando https://books.toscrape.com/catalogue/category/books/travel_2/index.html...
  📖 Procesando: It's Only the Himalayas
  📖 Procesando: Full Moon over Noahâs Ark: An Odyssey to Mount Ararat and Beyond
  📖 Procesando: See America: A Celebration of Our National Parks & Treasured Sites
  📖 Procesando: Vagabonding: An Uncommon Guide to the Art of Long-Term World Travel
  📖 Procesando: Under the Tuscan Sun
  📖 Procesando: A Summer In Europe
  📖 Procesando: The Great Railway Bazaar
  📖 Procesando: A Year in Provence (Provence #1)
  📖 Procesando: The Road to Little Dribbling: Adventures of an American in Britain (Notes From a Small Island #2)
  📖 Procesando: Neither Here nor There: Travels in Europe
  📖 Procesando: 1,000 Places to See Before You Die
✅ Categoría Travel completada

📚 Scrapeando categoría: Mystery
Scrapeando https://books.toscrape.com/catalogue/category/boo

# Las 5 consultas emocionales

In [ ]:
def ejecutar_consultas_challenge_completo():
 
    
    conn = sqlite3.connect("libros.db")
    cursor = conn.cursor()
    
    print("🕵️ INVESTIGACIÓN: VERDADES OSCURAS DE BOOKS TO SCRAPE")
    print("=" * 60)
    
    # 1. ¿Qué autor sigue publicando catástrofes de 1 estrella?
    print("\n💀 1. AUTORES QUE DEBERÍAN DEJAR DE ESCRIBIR")
    print("Comentario: Para identificar autores con consistentemente mal rating")
    print("¿Por qué importa? Ayuda a lectores a evitar decepciones y a editores a tomar mejores decisiones")
    
    cursor.execute("""
        SELECT a.nombre, COUNT(*) as libros_horribles,
               AVG(l.precio) as precio_promedio, --calcula el promedio
               GROUP_CONCAT(l.titulo, '; ') as titulos_desastrosos
        FROM autores a
        JOIN libro_autor la ON a.id = la.autor_id
        JOIN libros l ON la.libro_id = l.id
        WHERE l.rating = 'One'
        GROUP BY a.nombre
        HAVING COUNT(*) >= 2 --se usa para condiciones sobre agregadas
        ORDER BY libros_horribles DESC
        LIMIT 5
    """)
    
    resultados = cursor.fetchall()
    if resultados:
        for autor, cantidad, precio_prom, titulos in resultados:
            print(f"   😱 {autor}: {cantidad} desastres (£{precio_prom:.2f} promedio)")
            print(f"      Títulos: {titulos[:100]}...")
    else:
        print("   🎉 ¡Sorpresa! No hay autores consistentemente malos con 2+ libros")
    
    # 2. ¿Quién es el capo de la literatura barata pero bien rankeada?
    print("\n💎 2. REYES DE LA LITERATURA BARATA PERO BUENA")
    print("Comentario: Autores que ofrecen calidad sin quebrar la billetera")
    print("¿Por qué importa? Identifica oportunidades para lectores con presupuesto limitado")
    
    cursor.execute("""
        SELECT a.nombre, 
               AVG(l.precio) as precio_promedio, 
               COUNT(*) as total_libros,
               AVG(CASE l.rating 
                   WHEN 'Five' THEN 5 WHEN 'Four' THEN 4 WHEN 'Three' THEN 3
                   WHEN 'Two' THEN 2 WHEN 'One' THEN 1 ELSE 0 END) as rating_promedio,
               MIN(l.precio) as libro_mas_barato
        FROM autores a
        JOIN libro_autor la ON a.id = la.autor_id
        JOIN libros l ON la.libro_id = l.id
        WHERE l.precio < 25
        GROUP BY a.nombre
        HAVING COUNT(*) >= 2 AND rating_promedio >= 3.5
        ORDER BY rating_promedio DESC, precio_promedio ASC
        LIMIT 10
    """)
    
    resultados = cursor.fetchall()
    for autor, precio_prom, total, rating_prom, mas_barato in resultados:
        print(f"   👑 {autor}: £{precio_prom:.2f} promedio ({total} libros)")
        print(f"      Rating: {rating_prom:.1f}⭐ | Más barato: £{mas_barato:.2f}")
    
    # 3. Análisis de precios por categoría
    print("\n💰 3. CATEGORÍAS MÁS CARAS (Dónde se va tu plata)")
    print("Comentario: Identifica géneros más costosos para planificar presupuesto")
    print("¿Por qué importa? Ayuda a lectores a entender dinámicas de precios del mercado")
    
    cursor.execute("""
        SELECT c.nombre, 
               AVG(l.precio) as precio_promedio,
               MAX(l.precio) as precio_maximo,
               MIN(l.precio) as precio_minimo,
               COUNT(*) as total_libros,
               COUNT(CASE WHEN l.rating IN ('Four', 'Five') THEN 1 END) as libros_buenos
        FROM categorias c
        JOIN libros l ON c.id = l.categoria_id
        GROUP BY c.nombre
        HAVING COUNT(*) >= 3
        ORDER BY precio_promedio DESC
        LIMIT 10
    """)
    
    resultados = cursor.fetchall()
    for categoria, precio_prom, precio_max, precio_min, total, buenos in resultados:
        porcentaje_buenos = (buenos/total)*100 if total > 0 else 0
        print(f"   💸 {categoria}: £{precio_prom:.2f} promedio")
        print(f"      Rango: £{precio_min:.2f} - £{precio_max:.2f} | {porcentaje_buenos:.0f}% con 4+ estrellas")
    
    # 4. Joyas ocultas (precio bajo, rating alto)
    print("\n💍 4. JOYAS OCULTAS (Gran calidad, precio bajo)")
    print("Comentario: Libros con excelente rating pero precio accesible")
    print("¿Por qué importa? Para cuando estás en bancarrota pero con estándares altos")
    
    cursor.execute("""
        SELECT l.titulo, 
               COALESCE(a.nombre, 'Autor desconocido') as autor, 
               l.precio, l.rating, c.nombre as categoria,
               l.descripcion
        FROM libros l
        JOIN categorias c ON l.categoria_id = c.id
        LEFT JOIN libro_autor la ON l.id = la.libro_id
        LEFT JOIN autores a ON la.autor_id = a.id
        WHERE l.rating IN ('Four', 'Five') AND l.precio < 15
        ORDER BY 
            CASE l.rating WHEN 'Five' THEN 5 ELSE 4 END DESC, 
            l.precio ASC
        LIMIT 15
    """)
    
    resultados = cursor.fetchall()
    for titulo, autor, precio, rating, categoria, descripcion in resultados:
        # Toma solo los primeros 80 caracteres de la descipcion y añade puntos suspensivos si es mas larga
        desc_corta = descripcion[:80] + "..." if len(descripcion) > 80 else descripcion
        print(f"   💎 {titulo}")
        print(f"      {autor} | £{precio:.2f} | {rating}⭐ | {categoria}")
        print(f"      {desc_corta}")
        print()
    
    # 5. BONUS: ¿Por qué hay tantos libros amarillos? (Análisis de colores en títulos)
    print("\n🟡 5. EL MISTERIO DE LOS COLORES EN LOS TÍTULOS")
    print("Comentario: Análisis de frecuencia de colores mencionados en títulos")
    print("¿Por qué importa? Para entender patrones temáticos y de marketing literario")
    
    colores = ['Yellow', 'Red', 'Blue', 'Green', 'Black', 'White', 'Orange', 'Purple', 'Pink', 'Brown']
    
    for color in colores:
        cursor.execute("""
            SELECT COUNT(*) as cantidad,
                   AVG(precio) as precio_promedio,
                   GROUP_CONCAT(titulo, '; ') as ejemplos
            FROM libros 
            WHERE UPPER(titulo) LIKE ?
        """, (f'%{color.upper()}%',))
        
        resultado = cursor.fetchone()
        cantidad, precio_prom, ejemplos = resultado
        
        if cantidad > 0:
            ejemplos_cortos = ejemplos[:100] + "..." if len(ejemplos) > 100 else ejemplos
            print(f"   🎨 {color}: {cantidad} libros (£{precio_prom:.2f} promedio)")
            print(f"      Ejemplos: {ejemplos_cortos}")
    
    conn.close()

# Ejecutar consultas completas
ejecutar_consultas_challenge_completo()

🕵️ INVESTIGACIÓN: VERDADES OSCURAS DE BOOKS TO SCRAPE

💀 1. AUTORES QUE DEBERÍAN DEJAR DE ESCRIBIR
Comentario: Para identificar autores con consistentemente mal rating
¿Por qué importa? Ayuda a lectores a evitar decepciones y a editores a tomar mejores decisiones
   😱 Autor por determinar: 226 desastres (£34.56 promedio)
      Títulos: The Great Railway Bazaar; The Road to Little Dribbling: Adventures of an American in Britain (Notes ...
   😱 Product TypeBooks: 22 desastres (£22.03 promedio)
      Títulos: Tastes Like Fear (DI Marnie Rome #3); Her Backup Boyfriend (The Sorensen Family #1); The Purest Hook...
   😱 Natsuki Takaya: 6 desastres (£11.97 promedio)
      Títulos: Fruits Basket, Vol. 7 (Fruits Basket #7); Fruits Basket, Vol. 5 (Fruits Basket #5); Fruits Basket, V...
   😱 Bill Bryson: 6 desastres (£14.43 promedio)
      Títulos: The Road to Little Dribbling: Adventures of an American in Britain (Notes From a Small Island #2); T...
   😱 Paul Theroux: 4 desastres (£15.27 promedio

In [5]:
import sqlite3
import time
def demostrar_indexacion():
    """
    Demostración dramática del poder de los índices
    """
    print("\n⚡ DEMOSTRACIÓN DE INDEXACIÓN: ANTES Y DESPUÉS")
    print("=" * 60)
    
    conn = sqlite3.connect("libros.db")
    cursor = conn.cursor()
    
    # Primero, ELIMINAR todos los índices para empezar limpio
    print("🧹 Limpiando índices existentes...")
    cursor.execute("DROP INDEX IF EXISTS idx_libros_precio")
    cursor.execute("DROP INDEX IF EXISTS idx_libros_rating")
    cursor.execute("DROP INDEX IF EXISTS idx_libro_autor_libro")
    cursor.execute("DROP INDEX IF EXISTS idx_libro_autor_autor")
    cursor.execute("DROP INDEX IF EXISTS idx_libros_categoria")
    
    # Consulta compleja que será lenta sin índices
    consulta_compleja = """
        SELECT l.titulo, a.nombre as autor, l.precio, l.rating, c.nombre as categoria
        FROM libros l
        JOIN libro_autor la ON l.id = la.libro_id
        JOIN autores a ON la.autor_id = a.id
        JOIN categorias c ON l.categoria_id = c.id
        WHERE l.precio > 20 AND l.rating IN ('Four', 'Five')
        ORDER BY l.precio DESC, l.titulo
        LIMIT 50
    """
    
    print("\n🐌 ANTES: Sin índices (preparate para el sufrimiento)")
    start_time = time.time()
    cursor.execute(consulta_compleja)
    resultados_sin_indice = cursor.fetchall()
    tiempo_sin_indice = time.time() - start_time
    print(f"   ⏱️  Tiempo sin índices: {tiempo_sin_indice:.4f} segundos")
    print(f"   📊 Resultados encontrados: {len(resultados_sin_indice)}")
    
    # Crear índices estratégicos
    print("\n🔧 Creando índices mágicos...")
    indices = [
        ("idx_libros_precio", "CREATE INDEX idx_libros_precio ON libros(precio)"),
        ("idx_libros_rating", "CREATE INDEX idx_libros_rating ON libros(rating)"),
        ("idx_libro_autor_libro", "CREATE INDEX idx_libro_autor_libro ON libro_autor(libro_id)"),
        ("idx_libro_autor_autor", "CREATE INDEX idx_libro_autor_autor ON libro_autor(autor_id)"),
        ("idx_libros_categoria", "CREATE INDEX idx_libros_categoria ON libros(categoria_id)")
    ]
    
    for nombre, sql in indices:
        cursor.execute(sql)
        print(f"   ✅ Creado: {nombre}")
    
    print("\n🚀 DESPUÉS: Con índices (siente el poder)")
    start_time = time.time()
    cursor.execute(consulta_compleja)
    resultados_con_indice = cursor.fetchall()
    tiempo_con_indice = time.time() - start_time
    print(f"   ⏱️  Tiempo con índices: {tiempo_con_indice:.4f} segundos")
    print(f"   📊 Resultados encontrados: {len(resultados_con_indice)}")
    
    # Calcular mejora
    if tiempo_sin_indice > 0:
        mejora_porcentaje = ((tiempo_sin_indice - tiempo_con_indice) / tiempo_sin_indice) * 100
        factor_mejora = tiempo_sin_indice / tiempo_con_indice if tiempo_con_indice > 0 else "∞"
        print(f"\n🎯 MEJORA DE RENDIMIENTO:")
        print(f"   📈 {mejora_porcentaje:.1f}% más rápido")
        if isinstance(factor_mejora, (int, float)):
            print(f"   ⚡ {factor_mejora:.2f}x más veloz")
    
    # Mostrar algunos resultados
    print("\n📋 Algunos libros caros pero buenos encontrados:")
    for i, (titulo, autor, precio, rating, categoria) in enumerate(resultados_con_indice[:8]):
        print(f"   {i+1:2d}. {titulo} - {autor}")
        precio_float = float(precio.replace("£", "").replace("Â", "").strip())
        print(f"       £{precio_float:.2f} | {rating}⭐ | {categoria}")
    conn.close()

# Ejecutar demostración de indexación
demostrar_indexacion()


⚡ DEMOSTRACIÓN DE INDEXACIÓN: ANTES Y DESPUÉS
🧹 Limpiando índices existentes...

🐌 ANTES: Sin índices (preparate para el sufrimiento)
   ⏱️  Tiempo sin índices: 0.0161 segundos
   📊 Resultados encontrados: 50

🔧 Creando índices mágicos...
   ✅ Creado: idx_libros_precio
   ✅ Creado: idx_libros_rating
   ✅ Creado: idx_libro_autor_libro
   ✅ Creado: idx_libro_autor_autor
   ✅ Creado: idx_libros_categoria

🚀 DESPUÉS: Con índices (siente el poder)
   ⏱️  Tiempo con índices: 0.0194 segundos
   📊 Resultados encontrados: 50

🎯 MEJORA DE RENDIMIENTO:
   📈 -20.3% más rápido
   ⚡ 0.83x más veloz

📋 Algunos libros caros pero buenos encontrados:
    1. The Man Who Mistook His Wife for a Hat and Other Clinical Tales - Oliver Sacks
       £59.45 | Four⭐ | Nonfiction
    2. The Death of Humanity: and the Case for Life - Richard Weikart
       £58.11 | Four⭐ | Philosophy
    3. The White Cat and the Monk: A Retelling of the Poem âPangur BÃ¡nâ - Product TypeBooks
       £58.08 | Four⭐ | Childrens
 